In [3]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 7.9 MB/s eta 0:00:00


In [21]:
import requests
import re
from bs4 import BeautifulSoup
from pypdf import PdfReader
from tqdm import tqdm
from pprint import pprint
import os

COPYRIGHT_SET = {68, 70, 73, 80, 165, 179, 319, 328}

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text

def process_transcript(text):
    # Remove any additional copyright or note sections if necessary
    text = re.sub(r'Copyright.*?All Rights Reserved\.', '', text, flags=re.DOTALL)

    # Format the dialogue: remove excessive line breaks, ensure spacing between speakers
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)
    formatted_text = re.sub(r'([A-Za-z\s]+):', r'\n\1:', text)  # Ensure speaker names are on new lines
    formatted_text = re.sub(r'\n{2,}', '\n', formatted_text)  # Clean up multiple newlines
    formatted_text = re.sub(r'\n\s+([A-Za-z\s]+:)', r'\n\1', formatted_text)
    formatted_text = re.sub(r'[ \t]{2,}', ' ', formatted_text) # Clean up multiple spaces or tabs

    return formatted_text.strip()

def extract_text_from_web_transcript(content):
    try:
        unwanted_start = content.find(string=re.compile(r"Transcripts may contain a few typos"))
        unwanted_end = content.find('hr', class_='wp-block-separator')
        if unwanted_start and unwanted_end:
            # Find the index range of the unwanted part and remove it
            start_index = unwanted_start.parent
            for elem in start_index.find_next_siblings():
                if elem == unwanted_end:
                    break
                elem.extract()
            start_index.extract()
        text_content = content.get_text().split("Related")[0]

        formatted_text = re.sub(r"(?<=\n)([A-Za-z\s]+):", r"\1:", text_content)
        formatted_text = re.sub(r"\n{2,}", "\n\n", formatted_text).strip()
        return formatted_text
    except:
        return "transcript cannot be automatically extracted."

def get_all_transcript(url, pdf_folder="./content"):
    if not os.path.exists(pdf_folder):
        os.makedirs(pdf_folder)
    pattern = re.compile(r"#(\d+):")

    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    links = soup.find_all("a", string=pattern)

    link_data = []
    for link in tqdm(links):
        text = link.get_text()
        href = link.get("href")
        match = pattern.search(text)
        link_id = int(match.group(1))

        if link_id not in COPYRIGHT_SET:
            resp = requests.get(href)
            if not href.endswith("pdf"):
                soup = BeautifulSoup(resp.text, "html.parser")
                content = soup.find("div", class_="entry-content")
                transcript = extract_text_from_web_transcript(content)
            else:
                filename = os.path.join(pdf_folder, href.split("/")[-1])
                with open(filename, "wb") as f:
                    f.write(resp.content)
                transcript = extract_text_from_pdf(filename)
                transcript = process_transcript(transcript)
        else:
            transcript = "not available due to copyright reasons"


        link_dict = {
            "id": link_id,
            "title": text,
            "url": href,
            "transcript": transcript
        }

        # Append the dictionary to the list
        link_data.append(link_dict)

    return link_data

In [22]:
transcripts_url = "https://tim.blog/2018/09/20/all-transcripts-from-the-tim-ferriss-show/"
link_data = get_all_transcript(transcripts_url)

100%|██████████| 687/687 [12:42<00:00,  1.11s/it]


In [25]:
import json

with open("data.json", "w") as f:
    json.dump(link_data, f)